In [1]:
import praw
import pandas as pd
import datetime as dt
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from time import sleep
import pymongo
import numpy as np
import spacy
import tweepy
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from pprint import pprint

nlp = spacy.load('en')

C:\Users\APG\Anaconda3\envs\AngelosAwesomeEnvironment\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\Users\APG\Anaconda3\envs\AngelosAwesomeEnvironment\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
C:\Users\APG\Anaconda3\envs\AngelosAwesomeEnvironment\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\Users\APG\Anaconda3\envs\AngelosAwesomeEnvironment\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [2]:
def reddit():

    reddit = praw.Reddit(client_id='OxteSNrREDwJUw', \
                     client_secret='FeY9Yx1ouGLTTpZRRNWXdpGWTIM', \
                     user_agent='nw_project2_getcities', \
                     username='hawkcatdogbird', \
                     password='23Reddog!')
    return reddit       

In [3]:
def nlp_me(comment):
    #feed me text strings, I dance with spacy
    nlp_key = []
    nlp_value = []
    doc = nlp(comment)
    if not doc.ents:
        nlp_key.append("no key")
        nlp_value.append("no entry")
    else:
        for ent in doc.ents:
            if ent.text not in nlp_key:
                nlp_key.append(ent.text)
            elif ent.text not in nlp_value:
                nlp_value.append(ent.label_)
            else:
                pass
            
    return nlp_key, nlp_value

In [17]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas','California','Colorado', 'Connecticut','Delaware', 'Florida', 
'Georgia','Hawaii', 'Idaho', 'Illinois','Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 
'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana','Nebraska', 'Nevada', 'NewHampshire', 
'NewJersey', 'NewMexico', 'NewYork', 'NorthCarolina', 'NorthDakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 
'RhodeIsland', 'SouthCarolina', 'SouthDakota','Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 
'WestVirginia', 'Wisconsin', 'Wyoming', 'washingtondc']

city_subreddits = ['NYC', 'Seattle', 'Chicago', 'LosAngeles', 'Portland', 'Boston', 'Detroit',
'SanFrancisco', 'Atlanta', 'Philadelphia', "Columbus", 'Dallas', 'Baltimore', 'StLouis', 'SanDiego', 
'Houston', "Phoenix", "Orlando", "Miami"]

In [5]:
r = reddit()

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.project_reax

In [6]:
def scrape_state_sub(subredditname, period):

    words = ''
    headlines = []
    #pos
    posheadlines = []
    poskarma = []
    commentspos = []
    #neg
    negheadlines = []
    negkarma = []
    commentsneg = []
    #neu
    neuheadlines = []
    neukarma = []
    commentsneu = []

    headlinesentiment = []
    analyzer = SentimentIntensityAnalyzer()


    for submission in r.subreddit(subredditname).top(period):


        headlines.append(submission.title)

        words = words + " " + submission.title

        results = analyzer.polarity_scores(submission.title)

        cresults = results['compound']
        headlinesentiment.append(cresults)

        if (cresults > .2):
            posheadlines.append(cresults)
            poskarma.append(submission.score)
            commentspos.append(submission.num_comments)

        elif (cresults < (-0.2)):
            negheadlines.append(cresults)
            negkarma.append(submission.score)
            commentsneg.append(submission.num_comments)
        else:
            neuheadlines.append(cresults)
            neukarma.append(submission.score)
            commentsneu.append(submission.num_comments)

    headtot = len(headlines)
    headp = len(posheadlines)
    headneg = len(negheadlines)
    headneu = len(neuheadlines)

    percposhead = headp/headtot
    percneghead = headneg/headtot
    percneuhead = headneu/headtot
    
    posavgkarma = (np.sum(poskarma)/headp)
    negavgkarma = (np.sum(negkarma)/headneg)
    neuavgkarma = (np.sum(neukarma)/headneu)
    
    posavgcomms = (np.sum(commentspos)/headp)
    negavgcomms = (np.sum(commentsneg)/headneg)
    neuavgcomms = (np.sum(commentsneu)/headneu)

    cs = np.mean(headlinesentiment)

    tokenizer = RegexpTokenizer(r'\w+')
    stop_words = stopwords.words('english')


    text_tokens = []
    toks = tokenizer.tokenize(words)
    toks = [t.lower() for t in toks if t.lower() not in stop_words]
    text_tokens.extend(toks)

    text_frequency = nltk.FreqDist(text_tokens)

    t_freq = text_frequency.most_common(30)

    key_holder, value_holder = nlp_me(words)
#     print(key_holder)
       
    
    statedict = dict(location = subredditname,
                     headlines = headlines,
                     headlineCOMSENTAVG = cs,
                     percposhead = percposhead,
                     percneghead = percneghead,
                     percneuhead = percneuhead,
                     avgupscorePOSHL = posavgkarma,
                     avgupscoreNEGHL = negavgkarma,
                     avgupscoreNEUHL = neuavgkarma,
                     posavgcomms = posavgcomms,
                     negavgcomms = negavgcomms,
                     neuavgcomms = neuavgcomms,                   
                     POSheadlines = posheadlines,
                     NEGheadlines = negheadlines,
                     NEUheadlines = neuheadlines,
                     wordcloud = t_freq,
                     keys = key_holder)

    db.state_simple.insert_one(statedict)


In [18]:
def scrape_city_sub(subredditname, period):

    words = ''
    headlines = []
    #pos
    posheadlines = []
    poskarma = []
    commentspos = []
    #neg
    negheadlines = []
    negkarma = []
    commentsneg = []
    #neu
    neuheadlines = []
    neukarma = []
    commentsneu = []

    headlinesentiment = []
    analyzer = SentimentIntensityAnalyzer()


    for submission in r.subreddit(subredditname).top(period):


        headlines.append(submission.title)

        words = words + " " + submission.title

        results = analyzer.polarity_scores(submission.title)

        cresults = results['compound']
        headlinesentiment.append(cresults)

        if (cresults > .2):
            posheadlines.append(cresults)
            poskarma.append(submission.score)
            commentspos.append(submission.num_comments)

        elif (cresults < (-0.2)):
            negheadlines.append(cresults)
            negkarma.append(submission.score)
            commentsneg.append(submission.num_comments)
        else:
            neuheadlines.append(cresults)
            neukarma.append(submission.score)
            commentsneu.append(submission.num_comments)

    headtot = len(headlines)
    headp = len(posheadlines)
    headneg = len(negheadlines)
    headneu = len(neuheadlines)

    percposhead = headp/headtot
    percneghead = headneg/headtot
    percneuhead = headneu/headtot
    
    posavgkarma = (np.sum(poskarma)/headp)
    negavgkarma = (np.sum(negkarma)/headneg)
    neuavgkarma = (np.sum(neukarma)/headneu)
    
    posavgcomms = (np.sum(commentspos)/headp)
    negavgcomms = (np.sum(commentsneg)/headneg)
    neuavgcomms = (np.sum(commentsneu)/headneu)

    cs = np.mean(headlinesentiment)

    tokenizer = RegexpTokenizer(r'\w+')
    stop_words = stopwords.words('english')


    text_tokens = []
    toks = tokenizer.tokenize(words)
    toks = [t.lower() for t in toks if t.lower() not in stop_words]
    text_tokens.extend(toks)

    text_frequency = nltk.FreqDist(text_tokens)

    t_freq = text_frequency.most_common(30)

    key_holder, value_holder = nlp_me(words)
#     print(key_holder)
    
    if (subredditname == 'NYC'):
        subredditname = 'New York'
    else:
        subredditname = subredditname
        
    date = dt.datetime.now().strftime('%Y-%m-%d')
    
    statedict = dict(location = subredditname,
                     headlines = headlines,
                     headlineCOMSENTAVG = cs,
                     percposhead = percposhead,
                     percneghead = percneghead,
                     percneuhead = percneuhead,
                     avgupscorePOSHL = posavgkarma,
                     avgupscoreNEGHL = negavgkarma,
                     avgupscoreNEUHL = neuavgkarma,
                     posavgcomms = posavgcomms,
                     negavgcomms = negavgcomms,
                     neuavgcomms = neuavgcomms,                   
                     POSheadlines = posheadlines,
                     NEGheadlines = negheadlines,
                     NEUheadlines = neuheadlines,
                     wordcloud = t_freq,
                     keys = key_holder,
                     date = data)

    db.city_simple.insert_one(statedict)


In [10]:
month = 'month'

for state in states:

    scrape_state_sub(state, month)

In [19]:
day = 'day'

for city in city_subreddits:
    
    scrape_city_sub(city, day)